In [1]:
%matplotlib inline

In [2]:
import serial, threading, sys, time, os, datetime, warnings, numpy as np, matplotlib.pyplot as plt
from IPython.display import clear_output
from timeit import default_timer as timer
sys.path.append('/home/pi/Desktop/ADRNX/SRS')
import SIM900, SIM921, SIM922

In [3]:
mf = SIM900.SIM900('/home/pi/Desktop/ADRNX/config.yaml',debug = True)
mf.debug = True
mf.init_mainframe()
s922 = SIM922.SIM922(mf,'s922',debug=True);
#s921 = SIM921.SIM921(mf,'s921',debug=True);
mf.register_and_init_mods([s922])
mf.start_comm_threads()

SIM mainframe device: /dev/ADR_sim900
SIM 922 registered!
Output loop started
Input loop started
Input read thread is up!


In [ ]:
#4.341500E+01, 3.167000E+00, 3.186000E+00, 1.105380E+02 8pm j14
#4.333300E+01, 2.816000E+00, 2.793000E+00, 7.035300E+01 10am j15
#4.333600E+01, 2.602000E+00, 2.565000E+00, 1.137100E+01 2pm j16
#4.334200E+01, 2.592000E+00, 2.576000E+00, 5.787000E+00 4pm j16
#4.336400E+01, 2.591000E+00, 2.573000E+00, 2.602000E+00 10am j17
mfc.get_serial_object().write('*RST\n')

In [ ]:
print str(bin(int(mfc.process_cmd_with_resp('*STB?'))))
print mfc.process_cmd_with_resp('LEXE?')
print mfc.process_cmd_with_resp('LCME?')
print mfc.process_cmd_with_resp('MSGL?')
print mfc.process_cmd_with_resp('*ESR?')
print mfc.process_cmd_with_resp('*SRE?')
print mfc.process_cmd_with_resp('*ESE?')

In [ ]:
print mfc.process_cmd_with_resp('*IDN?')
print mfc.process_cmd_with_resp('ENSS?')
print mfc.process_cmd_with_resp('*PSC?')

In [ ]:
with warnings.catch_warnings():
    #warnings.simplefilter('error')
    for i in range(1,2):
        print mfc.process_cmd_with_resp('SEND 6,"TVAL? 0,1\n"')
        time.sleep(3)
        if i%5 == 0:
            clear_output()

In [ ]:
try:
    with warnings.catch_warnings():
        warnings.simplefilter('error')
        s922.start_data_stream(0)
        for i in range(1,200):        
            time.sleep(1)
            if i%10 == 0:
                clear_output()    
except Exception as e:
    print(e)
finally:
    s922.stop_data_stream()

In [ ]:
mfc.set_RPER([s922.MODNUM])

In [ ]:
print mfc.process_cmd_with_resp('SEND 6,"TERM 0\n"')

In [ ]:
print mfc.process_cmd_with_resp('TERM D,LF\n')
print mfc.process_cmd_with_resp('MSGL 128\n')

In [ ]:
mfc.get_serial_object().write('SRST 6\n')
mfc.get_serial_object().write('AINP? 6\n')
mfc.get_serial_object().write('AOUT? 6\n')

In [ ]:
len('4.333100E+01, 2.601000E+00, 2.590000E+00, 1.122700E+01')

In [ ]:

mfc.debug = True
s922.dbg = True
s922.start_data_stream(0)
i = 0
try:
    while timer() - s < (24*60*60):
        s922.newdataevent.clear()
        s922.newdataevent.wait(5)
        s922.newdataevent.clear()
        temp = s922.lastvalues
        tm = s922.lastdatatimes
                   
        sys.stdout.flush()
        sys.stderr.flush()
        try:
            print s922.excq.get_nowait()
            print "OOPS!"
        except:
            #print "No exceptions!"
            pass                
        #print("|{:5d}|{:s}|{:8f}".format(i,temp,tm[0]))
        print('{} {:10f} {:f} {:f} {:f} {}'.format(i,temp[0],temp[1],temp[2],temp[3],tm[0]))
        #fl.write('{} {} {}'.format(i,temp,tm[0]))
        temps.append(list(temp));
        times.append(list(tm));
        i+=1
        time.sleep(5)
except Exception as e:
    print e
finally:
    s922.stop_data_stream()
    s922.flush_queue()
    fl.close()

In [ ]:
mfc.stop_input_loop()
mfc.stop_output_loop()